In [1]:
import pathlib
import pandas as pd
import numpy as np
import pandas_profiling
import warnings
warnings.filterwarnings('ignore')
from mltools.mlcommon import one_hot_dataframe

In [2]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3)

In [3]:
project_directory = pathlib.Path().resolve().parent
data_directory  = project_directory.joinpath('data/raw')
report_directory  = project_directory.joinpath('reports')

In [4]:
train_file = 'train.csv'
test_file  = 'test.csv'
report_file = 'profile_report.html'
train_filepath = data_directory.joinpath(train_file)
test_filepath = data_directory.joinpath(test_file)
report_filepath = report_directory.joinpath(report_file)

In [5]:
def get_df_describe(df):
    describe_df  = df.describe(include='all').T
    describe_df['Null_Count']= df.isnull().sum()
    return describe_df

In [6]:
def has_cabin(deck_level):
    if (deck_level =='X'):
        return 'No'
    else: 
        return 'Yes'


In [7]:
def tweak_dataset(df):
    df['FamilySize'] = df["SibSp"] +df["Parch"] + 1
    df['Embarked'].fillna('S' ,inplace =True)
    df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.',expand =False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Sir', 'Jonkheer', 'Dona','Don'], 'Other')
    df['Title'] = df['Title'].replace(['Capt', 'Col', 'Dr', 'Major', 'Rev'],'Officer')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

    df['Cabin'].fillna('XXX' ,inplace=True)
    df['Deck_Level'] = df['Cabin'].apply(lambda x : x[0]) 
     
    df.drop(['Name','Ticket','Cabin',],inplace=True,axis =1) 
    #df = pd.get_dummies(df,drop_first=True)
    return df

In [8]:
train_df = pd.read_csv(train_filepath)
test_df = pd.read_csv(test_filepath)

In [9]:
pandas_profiling.ProfileReport(train_df).to_file(report_filepath)

In [10]:
train_df = tweak_dataset(train_df)
test_df = tweak_dataset(test_df)

In [11]:
median_ages =train_df.groupby(['Sex','Pclass','Title','SibSp','Parch'])['Age'].median()
 
def fill_missing_age(row):
    if pd.isnull(row['Age']):
        return median_ages[row['Sex'],row['Pclass'],row['Title'] ,row['SibSp'] ,row['Parch']]
    else:
        return row['Age']

train_df['Age']=train_df.apply(fill_missing_age,axis=1) 
 

median_ages_again =train_df.groupby(['Sex','Pclass','Title'])['Age'].median()
median_ages_again[median_ages_again.isnull()]

def fill_missing_age_again(row):
    if pd.isnull(row['Age']):
        return median_ages_again[row['Sex'],row['Pclass'],row['Title']]
    else:
        return row['Age']
    
train_df['Age']=train_df.apply(fill_missing_age_again,axis=1) 
test_df['Age']=test_df.apply(fill_missing_age_again,axis=1) 

In [16]:
get_df_describe(test_df)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Null_Count
PassengerId,418,NaN,NaN,NaN,1100.5,120.81,892,996.25,1100.5,1204.75,1309,0
Pclass,418,NaN,NaN,NaN,2.26555,0.841838,1,1,3,3,3,0
Sex,418,2,male,266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
Age,418,NaN,NaN,NaN,29.5766,13.2343,0.17,21.25,27,36.375,76,0
SibSp,418,NaN,NaN,NaN,0.447368,0.89676,0,0,0,1,8,0
Parch,418,NaN,NaN,NaN,0.392344,0.981429,0,0,0,0,9,0
Fare,417,NaN,NaN,NaN,35.6272,55.9076,0,7.8958,14.4542,31.5,512.329,1
Embarked,418,3,S,270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
FamilySize,418,NaN,NaN,NaN,1.83971,1.51907,1,1,1,2,11,0
Title,418,6,Mr,240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [13]:
Check

NameError: name 'Check' is not defined

In [17]:
train_df.fillna(-999,inplace=True)
test_df.fillna(-999,inplace=True)

In [18]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
FamilySize       int64
Title           object
Deck_Level      object
dtype: object

In [19]:
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,Title,Deck_Level
0,1,0,3,male,22.0,1,0,7.2500,S,2,Mr,X
1,2,1,1,female,38.0,1,0,71.2833,C,2,Mrs,C
2,3,1,3,female,26.0,0,0,7.9250,S,1,Miss,X
3,4,1,1,female,35.0,1,0,53.1000,S,2,Mrs,C
4,5,0,3,male,35.0,0,0,8.0500,S,1,Mr,X


In [20]:
Check

NameError: name 'Check' is not defined

In [21]:
full_dataset =pd.concat(objs=[train_df, test_df], axis=0,sort=True).reset_index(drop=True)  
full_dataset["Pclass"] = full_dataset["Pclass"].astype("str")

In [22]:
train_dataset =full_dataset[full_dataset['Survived'].notnull()]
train_y = full_dataset[full_dataset['Survived'].notnull()]['Survived']
test_dataset =full_dataset[full_dataset['Survived'].isnull()]
#train_dataset["Survived"] = train_dataset["Survived"].astype("int")
test_passengers = test_dataset['PassengerId']
train_dataset.drop(labels=["Survived",'PassengerId'],axis = 1,inplace=True)
test_dataset.drop(labels=["Survived",'PassengerId'],axis = 1,inplace=True)

In [23]:
cate_features_index = np.where(train_dataset.dtypes != float)[0]

In [24]:
skf = StratifiedKFold(n_splits=5)

In [25]:
from sklearn.model_selection  import  train_test_split

In [26]:
X_train, X_val, y_train, y_val = train_test_split(train_dataset,train_y,test_size=0.3, random_state=42,stratify = train_y)

In [27]:
from sklearn import metrics

In [28]:
from catboost import CatBoostClassifier, Pool, cv

In [29]:
catboost_model = CatBoostClassifier(loss_function='Logloss', custom_metric='AUC:hints=skip_train~false', metric_period=50)

In [30]:
catboost_model.fit(X_train, y_train,eval_set=(X_val, y_val),cat_features=cate_features_index)

Learning rate set to 0.074246
0:	learn: 0.6479511	test: 0.6533545	best: 0.6533545 (0)	total: 159ms	remaining: 2m 38s
50:	learn: 0.3890499	test: 0.4234668	best: 0.4234668 (50)	total: 2.86s	remaining: 53.3s
100:	learn: 0.3595710	test: 0.4166406	best: 0.4166406 (100)	total: 5.38s	remaining: 47.9s
150:	learn: 0.3067121	test: 0.4153031	best: 0.4153031 (150)	total: 9.1s	remaining: 51.2s
200:	learn: 0.2720220	test: 0.4220238	best: 0.4153031 (150)	total: 13.7s	remaining: 54.6s
250:	learn: 0.2507957	test: 0.4298607	best: 0.4153031 (150)	total: 18.6s	remaining: 55.4s
300:	learn: 0.2317035	test: 0.4330705	best: 0.4153031 (150)	total: 24.5s	remaining: 56.9s
350:	learn: 0.2144146	test: 0.4374050	best: 0.4153031 (150)	total: 30.9s	remaining: 57.2s
400:	learn: 0.2038210	test: 0.4394058	best: 0.4153031 (150)	total: 37s	remaining: 55.3s
450:	learn: 0.1952171	test: 0.4412159	best: 0.4153031 (150)	total: 41.9s	remaining: 51s
500:	learn: 0.1891056	test: 0.4467682	best: 0.4153031 (150)	total: 45.9s	remaini

In [ ]:
#cv_data = cv(Pool(X_train,y_train,cat_features=cate_features_index),catboost_model.get_params(),fold_count=10) 

In [32]:
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.metrics import confusion_matrix,roc_auc_score,mean_squared_error,f1_score,recall_score,precision_score

def model_evalution(model,x_train,y_train,x_test,y_test):
    print("####################### model Evalution started #######################")
    train_pre = model.predict(x_train)
    test_pre = model.predict(x_test)
    train_pro = model.predict_proba(x_train)
    test_pro = model.predict_proba(x_test)

    print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
    print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
    print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
    print("Train F1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
    print("Train recall: {0} \t Test recall: {1}".format(recall_score(y_train, train_pre),recall_score(y_test,test_pre)))
    print("Train precision: {0} \t Test Precision: {1}".format(precision_score(y_train, train_pre),precision_score(y_test,test_pre)))
    print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score

In [33]:
model_evalution(catboost_model,X_train, y_train,X_val,y_val)

####################### model Evalution started #######################
Train Accuracy: 0.8571428571428571 	 Test Accuracy: 0.832089552238806
Train Loss: 0.14285714285714285 	 Test Loss: 0.16791044776119404
Train AUC: 0.9215590132496514 	 Test AUC: 0.8706972639011474
Train F1: 0.8017817371937638 	 Test F1: 0.7643979057591622
Train recall: 0.7531380753138075 	 Test recall: 0.7087378640776699
Train precision: 0.8571428571428571 	 Test Precision: 0.8295454545454546
Train Confusion Matrix: 
[[354  30]
 [ 59 180]] 
 Test Confusion Matrix: 
[[150  15]
 [ 30  73]]


In [34]:
test_y =catboost_model.predict(test_dataset)

In [ ]:
submission =pd.DataFrame({'PassengerId':test_passengers.values ,'Survived':test_y}) 
import datetime
FORMAT = '%Y%m%d%H%M%S'
timestamp=datetime.datetime.now().strftime(FORMAT)
filename ="Titanic_CatBoost"+timestamp+"_out.csv"
#submission.to_csv(filename,index=False)

In [35]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
# Number of trees 
iterations = [int(x) for x in np.linspace(start = 100, stop = 500, num = 4)]
# Maximum number of levels in tree
depth = [int(x) for x in np.linspace(1, 10, num = 11)]
 
# Minimum number of samples required to split a node
l2_leaf_reg = [1,2,3,5,8]
# Minimum number of samples required at each leaf node
loss_function = ['Logloss','CrossEntropy']
# Method of selecting samples for training each tree
learning_rate =[0.005,0.03,0.1]
border_count =[32,]
ctr_border_count=[50,]
thread_count=4
eval_metric =["Precision"]
params = {'depth':depth,
          'iterations':iterations,
          'learning_rate':learning_rate, 
          'l2_leaf_reg':l2_leaf_reg,
          #'border_count':border_count,
          #'ctr_border_count':ctr_border_count,
          #'thread_count':thread_count,
          #'eval_metric':eval_metric,
         }
pprint(params)

{'depth': [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'iterations': [100, 233, 366, 500],
 'l2_leaf_reg': [1, 2, 3, 5, 8],
 'learning_rate': [0.005, 0.03, 0.1]}


In [36]:
from sklearn.model_selection import RandomizedSearchCV

In [38]:
cb_model = CatBoostClassifier()

randm = RandomizedSearchCV(estimator=cb_model, param_distributions = params, 
                               cv = 3, n_iter = 10, n_jobs=-1)
randm.fit(X_train, y_train,cat_features=cate_features_index)

0:	learn: 0.6914938	total: 52.4ms	remaining: 5.18s
1:	learn: 0.6900579	total: 121ms	remaining: 5.92s
2:	learn: 0.6886299	total: 178ms	remaining: 5.76s
3:	learn: 0.6872271	total: 217ms	remaining: 5.22s
4:	learn: 0.6858483	total: 299ms	remaining: 5.67s
5:	learn: 0.6842845	total: 371ms	remaining: 5.81s
6:	learn: 0.6827897	total: 393ms	remaining: 5.22s
7:	learn: 0.6812319	total: 440ms	remaining: 5.06s
8:	learn: 0.6796360	total: 470ms	remaining: 4.75s
9:	learn: 0.6782271	total: 542ms	remaining: 4.87s
10:	learn: 0.6765980	total: 581ms	remaining: 4.7s
11:	learn: 0.6751050	total: 638ms	remaining: 4.68s
12:	learn: 0.6736430	total: 694ms	remaining: 4.65s
13:	learn: 0.6722300	total: 729ms	remaining: 4.48s
14:	learn: 0.6706437	total: 785ms	remaining: 4.45s
15:	learn: 0.6692662	total: 824ms	remaining: 4.33s
16:	learn: 0.6678739	total: 863ms	remaining: 4.21s
17:	learn: 0.6664042	total: 912ms	remaining: 4.15s
18:	learn: 0.6649245	total: 944ms	remaining: 4.02s
19:	learn: 0.6636354	total: 992ms	remaini

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<catboost.core.CatBoostClassifier object at 0x000001EBC46FB160>,
                   iid='warn', n_iter=10, n_jobs=-1,
                   param_distributions={'depth': [1, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                  10],
                                        'iterations': [100, 233, 366, 500],
                                        'l2_leaf_reg': [1, 2, 3, 5, 8],
                                        'learning_rate': [0.005, 0.03, 0.1]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [39]:
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")


 Results from Random Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.0

 The best parameters across ALL searched params:
 {'learning_rate': 0.005, 'l2_leaf_reg': 8, 'iterations': 100, 'depth': 6}



In [ ]:
# catboost_model = CatBoostClassifier(iterations=400,
#                                     verbose=True,eval_metric="F1",
#                                     learning_rate=0.1,
#                                     class_weights=[1,3],
#                                     depth=3,l2_leaf_reg=3,bagging_temperature=0.5)

In [40]:
catboost_model = randm.best_estimator_

In [41]:
model_evalution(catboost_model,X_train, y_train,X_val,y_val)

####################### model Evalution started #######################
Train Accuracy: 0.8057784911717496 	 Test Accuracy: 0.7835820895522388
Train Loss: 0.1942215088282504 	 Test Loss: 0.21641791044776118
Train AUC: 0.8633902109483961 	 Test AUC: 0.8573109738158283
Train F1: 0.7397849462365591 	 Test F1: 0.712871287128713
Train recall: 0.7196652719665272 	 Test recall: 0.6990291262135923
Train precision: 0.7610619469026548 	 Test Precision: 0.7272727272727273
Train Confusion Matrix: 
[[330  54]
 [ 67 172]] 
 Test Confusion Matrix: 
[[138  27]
 [ 31  72]]


In [ ]:
test_y =catboost_model.predict(test_dataset)

In [ ]:
submission =pd.DataFrame({'PassengerId':test_passengers.values ,'Survived':test_y}) 
#submission['Loan_Status'].replace(0, 'N',inplace=True)
#submission['Loan_Status'].replace(1, 'Y',inplace=True)
import datetime
FORMAT = '%Y%m%d%H%M%S'
timestamp=datetime.datetime.now().strftime(FORMAT)
filename ="Titanic_CatBoost"+timestamp+"_out.csv"
#submission.to_csv(filename,index=False)

submission.to_csv(filename,index=False)